In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm


In [47]:
# Constants
hp = 10
enemy_dmg = 2
enemy_dodge = 0.2
enemy_shield = 1
ally_three_laser_damage = 1
ally_missle_damage = 2


In [70]:
array_damage_taken = []
array_count_laser_s = []
array_count_laser_p = []
array_count_laser_w = []
array_count_missle_n = []
array_count_missle_s = []
array_count_missle_p = []
array_count_missle_w = []
array_count_missle_first = []
array_count_use_missle_only_when_fail_to_break_room = []

for simulations in tqdm(range(10000)):

    # Main starting values
    damage_taken = 0
    count_laser_s = 0
    count_laser_p = 0
    count_laser_w = 0
    count_missle_n = 0
    count_missle_s = 0
    count_missle_p = 0
    count_missle_w = 0
    count_missle_first = 0
    count_use_missle_only_when_fail_to_break_room = 0
    # Other starting values
    hp = 10
    shield_room_down = False
    pilot_room_down = False
    weapon_room_down = False
    current_enemy_shield = enemy_shield
    current_enemy_dmg = enemy_dmg
    current_enemy_dodge = enemy_dodge
    max_iterations = 1000
    iterations = 0
    hit_max_iterations = False
    while hp > 0:
        iterations += 1
        if iterations > max_iterations:
            hit_max_iterations = True
            break
        if shield_room_down:
            current_enemy_shield = 0
            shield_room_down = False
        if pilot_room_down:
            current_enemy_dodge = 0
            pilot_room_down = False
        if weapon_room_down:
            current_enemy_dmg = 0
            weapon_room_down = False

        current_enemy_shield = enemy_shield
        broke_shield = False
        choice_laser = np.random.choice(['w', 's', 'p'])
        choice_missle = np.random.choice(['n','n','n','n','n','n','n','n','n','n', 's', 'p', 'w'])
        missle_first = np.random.choice([True, False])
        use_missle_only_when_fail_to_break_room = np.random.choice([True, False])

        # Update counts
        if choice_laser == 's':
            count_laser_s += 1
        elif choice_laser == 'p':
            count_laser_p += 1
        elif choice_laser == 'w':
            count_laser_w += 1
        if choice_missle == 'n':
            count_missle_n += 1
        elif choice_missle == 's':
            count_missle_s += 1
        elif choice_missle == 'p':
            count_missle_p += 1
        elif choice_missle == 'w':
            count_missle_w += 1
        if missle_first and choice_missle != 'n':
            count_missle_first += 1
        if use_missle_only_when_fail_to_break_room and choice_missle != 'n':
            count_use_missle_only_when_fail_to_break_room += 1



        # Main program
        if not missle_first: # Regular three laser first route
            for i in range(3):
                is_hit = 1 if np.random.uniform() > current_enemy_dodge else 0
                if is_hit:
                    current_enemy_shield -= 1
            if current_enemy_shield < 0:
                broke_shield = True
                hp -= (-1*ally_three_laser_damage*current_enemy_shield) # Damage incurred as well
            if broke_shield:
                if choice_laser == "s":
                    # Shield room taken down
                    shield_room_down = True
                    current_enemy_shield = 0
                elif choice_laser == "p":
                    # Pilot room taken down
                    pilot_room_down = True
                    current_enemy_dodge = 0
                elif choice_laser == "w":
                    # Weapons room taken down
                    weapon_room_down = True
                    current_enemy_dmg = 0
            # Missle step now
            if not use_missle_only_when_fail_to_break_room or not broke_shield:
                if choice_missle != "n": # N means none
                    for i in range(1):
                        is_hit = 1 if np.random.uniform() > current_enemy_dodge else 0
                        if is_hit:
                            broke_shield = True # Not truly broken but went past it anyway
                    if broke_shield:
                        hp -= (ally_missle_damage) # Damage incurred as well
                    if broke_shield:
                        if choice_missle == "s":
                            # Shield room taken down
                            shield_room_down = True
                            current_enemy_shield = 0
                        elif choice_missle == "p":
                            # Pilot room taken down
                            pilot_room_down = True
                            current_enemy_dodge = 0
                        elif choice_missle == "w":
                            # Weapons room taken down
                            weapon_room_down = True
                            current_enemy_dmg = 0
        else:
            # Missle first route
            if choice_missle != "n": # N means none
                for i in range(1):
                    is_hit = 1 if np.random.uniform() > current_enemy_dodge else 0
                    if is_hit:
                        broke_shield = True # Not truly broken but went past it anyway
                if broke_shield:
                    hp -= (ally_missle_damage) # Damage incurred as well
                if broke_shield:
                    if choice_missle == "s":
                        # Shield room taken down
                        shield_room_down = True
                        current_enemy_shield = 0
                    elif choice_missle == "p":
                        # Pilot room taken down
                        pilot_room_down = True
                        current_enemy_dodge = 0
                    elif choice_missle == "w":
                        # Weapons room taken down
                        weapon_room_down = True
                        current_enemy_dmg = 0
            # Laser step now
            for i in range(3):
                is_hit = 1 if np.random.uniform() > current_enemy_dodge else 0
                if is_hit:
                    current_enemy_shield -= 1
            if current_enemy_shield < 0:
                broke_shield = True
                hp -= (-1*ally_three_laser_damage*current_enemy_shield) # Damage incurred as well
            if broke_shield:
                if choice_laser == "s":
                    # Shield room taken down
                    shield_room_down = True
                    current_enemy_shield = 0
                elif choice_laser == "p":
                    # Pilot room taken down
                    pilot_room_down = True
                    current_enemy_dodge = 0
                elif choice_laser == "w":
                    # Weapons room taken down
                    weapon_room_down = True
                    current_enemy_dmg = 0


        # Take damage from enemey here
        damage_taken += current_enemy_dmg

        # Ending of a new turn (If it is still down it will be reset to 0 next turn; but just setting back for now)
        current_enemy_dodge = enemy_dodge
        current_enemy_shield = enemy_shield
        current_enemy_dmg = enemy_dmg

    if not hit_max_iterations:    
        array_damage_taken.append(damage_taken)
        array_count_laser_s.append(count_laser_s)
        array_count_laser_p.append(count_laser_p)
        array_count_laser_w.append(count_laser_w)
        array_count_missle_n.append(count_missle_n)
        array_count_missle_s.append(count_missle_s)
        array_count_missle_p.append(count_missle_p)
        array_count_missle_w.append(count_missle_w)
        array_count_missle_first.append(count_missle_first)
        array_count_use_missle_only_when_fail_to_break_room.append(count_use_missle_only_when_fail_to_break_room)
df = pd.DataFrame()
df['damage_taken'] = array_damage_taken
df['count_laser_s'] = array_count_laser_s
df['count_laser_p'] = array_count_laser_p
df['count_laser_w'] = array_count_laser_w
df['count_missle_n'] = array_count_missle_n
df['count_missle_s'] = array_count_missle_s
df['count_missle_p'] = array_count_missle_p
df['count_missle_w'] = array_count_missle_w
df['count_missle_first'] = array_count_missle_first
df['count_use_missle_only_when_fail_to_break_room'] = array_count_use_missle_only_when_fail_to_break_room


import xgboost as xgb
import shap

def get_final_score(count_missle_s, count_missle_p, count_missle_w, damage_taken):
    return damage_taken + 3*(count_missle_s + count_missle_p + count_missle_w)
df['final_score'] = df[['count_missle_s', 'count_missle_p', 'count_missle_w', 'damage_taken']].apply(lambda x: get_final_score(*x), axis='columns')



y = df['final_score']
features = [column for column in df.columns if column not in ['damage_taken', 'final_score']]
X = df[features]
model = xgb.XGBRegressor()
model.fit(X,y)

importance_scores = model.feature_importances_

# Print feature importance scores
for (i, score), feature in zip(enumerate(importance_scores), features):
    print(f"Feature {i}: {score} {feature}")


print("This is the tendency for the best score")
df.sort_values('final_score').iloc[0:50].sum()


100%|██████████| 10000/10000 [00:02<00:00, 4695.87it/s]


Feature 0: 0.100416399538517 count_laser_s
Feature 1: 0.0910143330693245 count_laser_p
Feature 2: 0.2592286169528961 count_laser_w
Feature 3: 0.0378563217818737 count_missle_n
Feature 4: 0.13751719892024994 count_missle_s
Feature 5: 0.12236714363098145 count_missle_p
Feature 6: 0.06053779274225235 count_missle_w
Feature 7: 0.028412330895662308 count_missle_first
Feature 8: 0.16264982521533966 count_use_missle_only_when_fail_to_break_room
This is the tendency for the best score


damage_taken                                       0
count_laser_s                                     56
count_laser_p                                     56
count_laser_w                                    202
count_missle_n                                   314
count_missle_s                                     0
count_missle_p                                     0
count_missle_w                                     0
count_missle_first                                 0
count_use_missle_only_when_fail_to_break_room      0
final_score                                        0
dtype: int64